In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv('./clean_train.csv')

X = df[['lat','lon','bathrooms','l2','l3','property_type','description']]
y = df['avg']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

algoritmo = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print(matriz)
recall = recall_score(y_test, y_pred)
print(recall)



[[28425  1779]
 [ 2525  6781]]
0.7286696754781861


In [2]:
from sklearn import preprocessing


columns2 = ['lat','lon','rooms','bedrooms','bathrooms','l2','l3','property_type', 'description']

df2 = pd.read_csv('./properties_colombia_test.csv')
df2 = df2[columns2]

def main_normalize(df, columns):
    
    # normalizando description
    df['description'] = df['description'].astype('str')
    df['description'] = df['description'].str.lower()
    df['description'] = df['description'].str.replace('|', '')
    df['description'] = df['description'].str.replace('<br>', '')
    df['description'] = df['description'].str.replace('<br />', '')
    df['description'] = df['description'].str.replace('\n', '')
    df['description'] = df['description'].str.replace('<b>', '')
    df['description'] = df['description'].str.replace('</b>', '')

    def normalize(s):
        replacements = (
            ("á", "a"),
            ("é", "e"),
            ("í", "i"),
            ("ó", "o"),
            ("ú", "u"),
        )
        for a, b in replacements:
            s = s.replace(a, b).replace(a.upper(), b.upper())
        return s

    df['description'] = df['description'].apply(lambda x: normalize(x))

    import html

    df['description'] = df['description'].apply(lambda x: html.unescape(x))
    
    # seleccionando las columnas que tienen valores adecuados
    df = df[columns]
       
    return df

new_test = main_normalize(df2, columns2)

C:\Users\KhorneFlakes\AppData\Local\Temp\ipykernel_3092\3440974376.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['description'] = df['description'].str.replace('|', '')


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

test_normalize = new_test.apply(lambda series: pd.Series(
        LabelEncoder().fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index
    ))


imputer = KNNImputer(n_neighbors=100)
test_transform = imputer.fit_transform(test_normalize)

from sklearn import preprocessing

datos_min_max2 = preprocessing.MinMaxScaler().fit_transform(test_transform)

df_clean_test = pd.DataFrame(datos_min_max2, columns=columns2)
df_clean_test

,lat,lon,rooms,bedrooms,bathrooms,l2,l3,property_type,description
0,0.412905,0.785461,0.0800,0.096774,0.111111,0.400000,0.091954,0.142857,0.530248
1,0.729034,0.377145,0.0932,0.097742,0.111111,0.033333,0.256705,0.000000,0.704332
2,0.801830,0.266513,0.0956,0.108710,0.000000,0.033333,0.517241,0.142857,0.277816
3,0.742745,0.410172,0.0948,0.114516,0.039444,0.033333,0.517241,0.571429,0.378595
4,0.497834,0.290041,0.0872,0.102581,0.055556,0.033333,0.517241,0.142857,0.387605
...,...,...,...,...,...,...,...,...,...
65845,0.754232,0.308661,0.0800,0.096774,0.107222,0.033333,0.517241,0.000000,0.235403
65846,0.550459,0.463342,0.0900,0.094839,0.000000,0.200000,0.494253,0.000000,0.826106
65847,0.604425,0.235510,0.0932,0.104516,0.055556,0.200000,0.578544,0.857143,0.272984
65848,0.889973,0.946706,0.0992,0.099355,0.055556,0.866667,0.099617,0.000000,0.861598


In [4]:

df_clean_test[['lat','lon','bathrooms','l2','l3','property_type', 'description']].to_csv('clean_test.csv', index=False)

In [5]:
df_test_respaldo = pd.read_csv('./clean_test.csv')
y_pred_test = algoritmo.predict(df_test_respaldo)
y_pred_test

array([1., 0., 0., ..., 0., 0., 0.])

In [6]:
df_test_pred = pd.DataFrame()
df_test_pred['pred'] = y_pred_test
df_test_pred.to_csv('khorneflakes-dev.csv', index=False)